<a href="https://colab.research.google.com/github/Rakshitkulkarni223/Content-Similarity/blob/main/Content_Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2022-02-04 16:59:20--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.162.240
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.162.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/root/input/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  99.9MB/s    in 21s     

2022-02-04 16:59:42 (74.2 MB/s) - ‘/root/input/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [4]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2022-02-04 17:04:34--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-02-04 17:04:34--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-02-04 17:04:34--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [5]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [7]:
import numpy as np
print('Indexing word vectors.')

embeddings_index = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 400000 word vectors.


In [3]:
EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz'
model_w2v = gensim.models.KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

In [33]:
import json
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import gensim

In [9]:
from google.colab import files
uploaded = files.upload()

Saving Dataset.json to Dataset.json


In [10]:
caption_comments_replies=open("Dataset.json")

In [11]:
caption_comments_replies=json.load(caption_comments_replies)

In [12]:
fst_caption=caption_comments_replies[1]['content']

In [13]:
fst_caption_comments=caption_comments_replies[1]['comments_and_replies']

In [14]:
all_comments_and_relpies=[]

for i in fst_caption_comments:
    all_comments_and_relpies.append(i['comment'])
    all_comments_and_relpies.extend(i['replies'])

In [15]:
def tokenization(text):
    tokens = word_tokenize(text)
    return tokens

In [16]:
def Capitalization(text):
    text=text.lower()
    return text

In [19]:
def remove_specail_chars(text):
    text=' '.join(text)
    tokenizer=nltk.RegexpTokenizer(r"\w+")
    text = tokenizer.tokenize(text)
    return text

In [20]:
def text_cleaner(text):
    rules = [
        {r'>\s+': u'>'},  # remove spaces after a tag opens or closes
        {r'\s+': u' '},  # replace consecutive spaces
        {r'\s*<br\s*/?>\s*': u'\n'},  # newline after a <br>
        {r'</(div)\s*>\s*': u'\n'},  # newline after </p> and </div> and <h1/>...
        {r'</(p|h\d)\s*>\s*': u'\n\n'},  # newline after </p> and </div> and <h1/>...
        {r'<head>.*<\s*(/head|body)[^>]*>': u''},  # remove <head> to </head>
        {r'<a\s+href="([^"]+)"[^>]*>.*</a>': r'\1'},  # show links instead of texts
        {r'[ \t]*<[^<]*?/?>': u''},  # remove remaining tags
        {r'^\s+': u''}  # remove spaces at the beginning
    ]
    for rule in rules:
        for (k, v) in rule.items():
            regex = re.compile(k)
            text = regex.sub(v, text)
            text=re.sub('https://[^ ]+','<link>',text)
            text=re.sub('http://[^ ]+','<link>',text)
            text=re.sub('@[^ ]+','<username>',text)
            text=re.sub('#[^ ]+','<hashtag>',text)
    text = text.rstrip()
    return Capitalization(text)

In [21]:
def Lemmatizing(filtered_sentence):
    
    filtered_sentence=remove_specail_chars(filtered_sentence)
    lemmatizer = WordNetLemmatizer()
    
    stemmed_Lemmatized_text=[]

    for word in filtered_sentence:
        word=lemmatizer.lemmatize(word)
        stemmed_Lemmatized_text.append(word)
        
    return stemmed_Lemmatized_text

In [34]:
def remove_stopwords(text):

    stop_words = set(stopwords.words('english'))
    
    cleaned_text=text_cleaner(text)
    
    word_tokens = tokenization(cleaned_text)

    filtered_sentence = [w for w in word_tokens if not w in stop_words]

    filtered_sentence = []

    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
            
    filtered_sentence=Lemmatizing(filtered_sentence)
    
    filtered_sentenc=remove_specail_chars(filtered_sentence)

    return filtered_sentence
    

In [31]:
documents=[]

In [40]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [41]:
filtered_caption=remove_stopwords(fst_caption)
documents.append(' '.join(filtered_caption))

In [42]:
filtered_comments=[]
for i in all_comments_and_relpies:
    filtered_comments.append(remove_stopwords(i))

In [43]:
for i in filtered_comments:
    documents.append(' '.join(i))

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
import pandas as pd


documents_df=pd.DataFrame(documents,columns=['documents'])


documents_df['documents_cleaned']=documents_df.documents

print(documents_df.documents)

tfidfvectoriser=TfidfVectorizer()
tfidfvectoriser.fit(documents_df.documents_cleaned)
tfidf_vectors=tfidfvectoriser.transform(documents_df.documents_cleaned)

pairwise_similarities=np.dot(tfidf_vectors,tfidf_vectors.T).toarray()
pairwise_differences=euclidean_distances(tfidf_vectors)

def most_similar(doc_id,similarity_matrix,matrix):
    print (f'Document: {documents_df.iloc[doc_id]["documents"]}')
    print ('\n')
    print ('Similar Documents:')
    if matrix=='Cosine Similarity':
        similar_ix=np.argsort(similarity_matrix[doc_id])[::-1]
    elif matrix=='Euclidean Distance':
        similar_ix=np.argsort(similarity_matrix[doc_id])
    for ix in similar_ix:
        if ix==doc_id:
            continue
        print('\n')
        print (f'Document: {documents_df.iloc[ix]["documents"]}')
        print (f'{matrix} : {similarity_matrix[doc_id][ix]}')

most_similar(0,pairwise_similarities,'Cosine Similarity')
most_similar(0,pairwise_differences,'Euclidean Distance')  

0      two men 17 year old girl found guilty murderin...
1      let u hope sentence justifies action unbelieva...
2                       valerie wheeler 17 year old girl
3      valerie wheeler s unbelievable country way thi...
4                 unbelievable ve ever encountered chavs
                             ...                        
139                                    wtf people wicked
140                                                cruel
141                                                 life
142                                absolutely disgusting
143                                             horrific
Name: documents, Length: 144, dtype: object
Document: two men 17 year old girl found guilty murdering doctor homophobic attack park centre cardiff bbc com three guilty homophobic murder doctor


Similar Documents:


Document: good found guilty sentence
Cosine Similarity : 0.34313213720196023


Document: valerie wheeler 17 year old girl
Cosine Similarity : 0.2350582003240

In [46]:
from gensim.models.keyedvectors import KeyedVectors
from gensim.models.doc2vec import TaggedDocument

In [47]:
import numpy as np


class DocSim:
    def __init__(self, w2v_model, stopwords=None):
        self.w2v_model = w2v_model
        self.stopwords = stopwords if stopwords is not None else []

    def vectorize(self, doc: str) -> np.ndarray:
        """
        Identify the vector values for each word in the given document
        :param doc:
        :return:
        """
        doc = doc.lower()
        words = [w for w in doc.split(" ") if w not in self.stopwords]
        word_vecs = []
        for word in words:
            try:
                vec = self.w2v_model[word]
                word_vecs.append(vec)
            except KeyError:
                # Ignore, if the word doesn't exist in the vocabulary
                pass

        # Assuming that document vector is the mean of all the word vectors
        # PS: There are other & better ways to do it.
        vector = np.mean(word_vecs, axis=0)
        return vector

    def _cosine_sim(self, vecA, vecB):
        """Find the cosine similarity distance between two vectors."""
        csim = np.dot(vecA, vecB) / (np.linalg.norm(vecA) * np.linalg.norm(vecB))
        if np.isnan(np.sum(csim)):
            return 0
        return csim

    def calculate_similarity(self, source_doc, target_docs=None, threshold=0):
        """Calculates & returns similarity scores between given source document & all
        the target documents."""
        if not target_docs:
            return []

        if isinstance(target_docs, str):
            target_docs = [target_docs]

        source_vec = self.vectorize(source_doc)
        results = []
        for doc in target_docs:
            target_vec = self.vectorize(doc)
            sim_score = self._cosine_sim(source_vec, target_vec)
            if sim_score > threshold:
                results.append({"score": sim_score, "doc": doc})
            # Sort results by score in desc order
            results.sort(key=lambda k: k["score"], reverse=True)

        return results

In [51]:
ds = DocSim(model_w2v)

# ds = DocSim(embeddings_index)

source_doc = fst_caption
target_docs = documents[1:]

sim_scores = ds.calculate_similarity(source_doc, target_docs)

print(fst_caption)
print(sim_scores)

Two men and a 17-year-old girl have been found guilty of murdering a doctor in a homophobic attack in a park in the centre of Cardiff.

BBC.COM
Three guilty of homophobic murder of doctor
[{'score': 0.6817749, 'doc': 'doctor saving life murderer killing must get sevior punishment'}, {'score': 0.6736083, 'doc': 'stamping anyone head hate crime sentence life matter victim never able walk free victim listen said beating someone death'}, {'score': 0.6699271, 'doc': 'happened poor man s pity teenage girl spend rest life jail'}, {'score': 0.66942346, 'doc': 'life life knowingly inflicted terrible injury innocent man proof undeniable death penalty needed case like beyond reasonable doubt imagine poor man went attack try justify still alive verdict call animal insulting member animal kingdom place society example made sadly wo n t terminal example'}, {'score': 0.6344825, 'doc': 'good found guilty sentence'}, {'score': 0.61762536, 'doc': 'carolyn bury sadly sweden 18 year old get adult prison g

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [49]:
target_docs

['let u hope sentence justifies action unbelievable 16 year old girl involved',
 'valerie wheeler 17 year old girl',
 'valerie wheeler s unbelievable country way thing going sadly',
 'unbelievable ve ever encountered chavs',
 'valerie wheeler forget half good behaviour',
 'paul evans okay okay',
 'jonas biwi dionne timms williams creigiau 16 time',
 'stamping anyone head hate crime sentence life matter victim never able walk free victim listen said beating someone death',
 'receive whole life jail sentence anything le joke far soft country serious crime',
 'gill robinson well said unless whole life joke life sentence halved good behaviour',
 'throw bloody key away gould never released community poor man',
 'one 17 enjoy rest youth early middle year prison bet n t think good idea r i p',
 'rob wilson sure work uk convicted adult',
 'jonas kalén think jonas may wrong',
 'jonas kalén judge great letting named thought big enough crime let time hope kick',
 'rob wilson read child young 10 t